# How to visualize your graph
This guide walks through how to visualize the graphs you create. This works with ANY Graph.

## Setup

## Set up Graph
You can visualize any arbitrary Graph, including StateGraph. Let's have some fun by drawing fractals :).

In [11]:
import random
from typing import Annotated, Literal

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]


class MyNode:
    def __init__(self, name: str):
        self.name = name

    def __call__(self, state: State):
        return {"messages": [("assistant", f"Called node {self.name}")]}


def route(state) -> Literal["entry_node", "__end__"]:
    if len(state["messages"]) > 10:
        return "__end__"
    return "entry_node"


def add_fractal_nodes(builder, current_node, level, max_level):
    if level > max_level:
        return

    # Number of nodes to create at this level
    num_nodes = random.randint(1, 3)  # Adjust randomness as needed
    for i in range(num_nodes):
        nm = ["A", "B", "C"][i]
        node_name = f"node_{current_node}_{nm}"
        builder.add_node(node_name, MyNode(node_name))
        builder.add_edge(current_node, node_name)

        # Recursively add more nodes
        r = random.random()
        if r > 0.2 and level + 1 < max_level:
            add_fractal_nodes(builder, node_name, level + 1, max_level)
        elif r > 0.05:
            builder.add_conditional_edges(node_name, route, node_name)
        else:
            # End
            builder.add_edge(node_name, "__end__")


def build_fractal_graph(max_level: int):
    builder = StateGraph(State)
    entry_point = "entry_node"
    builder.add_node(entry_point, MyNode(entry_point))
    builder.add_edge(START, entry_point)

    add_fractal_nodes(builder, entry_point, 1, max_level)

    # Optional: set a finish point if required
    builder.add_edge(entry_point, END)  # or any specific node

    return builder.compile()


app = build_fractal_graph(5)

## Mermaid
We can also convert a graph class into Mermaid syntax

In [12]:
print(app.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	entry_node(entry_node)
	node_entry_node_A(node_entry_node_A)
	node_entry_node_B(node_entry_node_B)
	node_node_entry_node_B_A(node_node_entry_node_B_A)
	node_node_node_entry_node_B_A_A(node_node_node_entry_node_B_A_A)
	node_node_entry_node_B_B(node_node_entry_node_B_B)
	node_node_node_entry_node_B_B_A(node_node_node_entry_node_B_B_A)
	node_node_node_entry_node_B_B_B(node_node_node_entry_node_B_B_B)
	node_node_node_node_entry_node_B_B_B_A(node_node_node_node_entry_node_B_B_B_A)
	node_node_node_node_entry_node_B_B_B_B(node_node_node_node_entry_node_B_B_B_B)
	node_node_entry_node_B_C(node_node_entry_node_B_C)
	node_node_node_entry_node_B_C_A(node_node_node_entry_node_B_C_A)
	node_node_node_entry_node_B_C_B(node_node_node_entry_node_B_C_B)
	node_node_node_node_entry_node_B_C_B_A(node_node_node_node_entry_node_B_C_B_A)
	node_node_node_node_entry_node_B_C_B_B(node_node_node_node_entry_node_B_C_B_B

## PNG
If preferred, we could render the Graph into a .png. Here we could use three options:

*   Using Mermaid.ink API (does not require additional packages)
*   Using Mermaid + Pyppeteer (requires pip install pyppeteer)
*   Using graphviz (which requires pip install graphviz)

### Using Mermaid.lnk
By default, draw_mermaid_png() uses Mermaid.Ink's API to generate the diagram.

In [19]:
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

display(
    Image(
        app.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

ReadTimeout: HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)

### Using Mermaid + Pyppeteer

In [8]:
%%capture --no-stderr
%pip install --quiet pyppeteer
%pip install --quiet nest_asyncio

In [15]:
import nest_asyncio

nest_asyncio.apply()  # Required for Jupyter Notebook to run async functions

display(
    Image(
        app.get_graph().draw_mermaid_png(
            curve_style=CurveStyle.LINEAR,
            node_colors=NodeStyles(first="#ffdfba", last="#baffc9", default="#fad7de"),
            wrap_label_n_words=9,
            output_file_path=None,
            draw_method=MermaidDrawMethod.PYPPETEER,
            background_color="white",
            padding=10,
        )
    )
)

[INFO] Starting Chromium download.


OSError: Chromium downloadable not found at https://storage.googleapis.com/chromium-browser-snapshots/Win_x64/1181205/chrome-win.zip: Received <?xml version='1.0' encoding='UTF-8'?><Error><Code>NoSuchKey</Code><Message>The specified key does not exist.</Message><Details>No such object: chromium-browser-snapshots/Win_x64/1181205/chrome-win.zip</Details></Error>.


### Use Graphviz

In [18]:
%%capture --no-stderr
%pip install pygraphviz

  error: subprocess-exited-with-error
  
  × Building wheel for pygraphviz (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [94 lines of output]
      C:\Users\gs\AppData\Local\Temp\pip-build-env-rrfkvl8p\overlay\Lib\site-packages\setuptools\config\_apply_pyprojecttoml.py:82: SetuptoolsDeprecationWarning: `project.license` as a TOML table is deprecated
      !!
      
              ********************************************************************************
              Please use a simple string containing a SPDX expression for `project.license`. You can also use `project.license-files`. (Both options available on setuptools>=77.0.0).
      
              By 2026-Feb-18, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              *************************************************

In [ ]:
try:
    display(Image(app.get_graph().draw_png()))
except ImportError:
    print(
        "You likely need to install dependencies for pygraphviz, see more here https://github.com/pygraphviz/pygraphviz/blob/main/INSTALL.txt"
    )